In [2]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [36]:
%sendtofile main.py

import time, machine
from oled_driver import oledshowfattext

if machine.wake_reason() != machine.PIN_WAKE:
    for i in range(10, 0, -1):
        oledshowfattext(["Ext Wake", str(i)])
        time.sleep(1)

oledshowfattext("DashButt")
time.sleep(1)

pbutton = machine.Pin(13, machine.Pin.IN, machine.Pin.PULL_DOWN)
v1 = pbutton.value()

oledshowfattext(["DashButt", str(v1)])
time.sleep(1)

pbutton.irq(trigger=machine.Pin.WAKE_HIGH if v1 == 0 else machine.Pin.WAKE_LOW, wake=machine.DEEPSLEEP)
machine.deepsleep()


Sent 19 lines (516 bytes) to main.py.


In [37]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.4-498-g5cd2c7f2e on 2018-09-08; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7efe59452240, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [30]:
%sendtofile --binary --source ../Sensor_Kennel/OLED_driver.py

Sent 3368 bytes in 113 chunks to OLED_driver.py.


In [45]:
%sendtofile /main.py

watercoolercode = """
from sevensegmentdisplay import encodeledstring, writeledstringautoscroll
import time

def phrases():
    for h in range(100, 110):
        yield 6, "%d fresh" % h
    yield 3600, "fresh"
    for h in range(1, 4):
        yield 3600, "fresh %d hours" % h
    for h in range(1, 4):
        yield 3600, "fresh %d hours" % h
    yield 3600, "stale"
    for h in range(4, 24):
        yield 3600, "stale %d hours" % h
    for d in range(1, 100):
        yield 24*3600, "bad %d days old" % d

@asyn.cancellable
async def setup():
    for t, ph in phrases():
        print(ph)
        encodeledstring(ph)
        t0 = time.time()
        while time.time() - t0 < t:
            writeledstringautoscroll()
            await asyn.sleep(0.1)
        
@asyn.cancellable
async def loop():
    pass
"""

import time
def sendscriptletcode(mqtt, channel, code):
    nbytes = 50
    npieces = int(len(code)/nbytes+1)
    print("Sending code in", npieces, "pieces")
    for i in range(npieces):
        time.sleep(0.5)  # give it a chance to queue
        mqtt.publish("scriptlet1/code", "%03d/%03d]%s" % (i+1, npieces, code[i*nbytes:(i+1)*nbytes]))


Sent 41 lines (1156 bytes) to /main.py.


In [46]:
%sendtofile --append /main.py

from oled_driver import oledshowfattext
oledshowfattext("DashButt")

import network
staif = network.WLAN(network.STA_IF)
staif.active(True)
staif.connect("DoESLiverpool", "decafbad00")
while not staif.isconnected():
    time.sleep(0.2)
oledshowfattext("MQTT")

from umqtt.robust import MQTTClient
mqttclient = MQTTClient(b"scriptlet1", "10.0.31.119")
mqttclient.connect()
mqttclient.publish(b"scriptlet1/ipbutton", staif.ifconfig()[0].encode())

oledshowfattext("sending")
sendscriptletcode(mqttclient, "scriptlet1/code", watercoolercode)
oledshowfattext("done.")
time.sleep(2)

import machine
machine.deepsleep()


Sent 23 lines (614 bytes) to /main.py.


In [53]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [54]:
%sendtofile /main.py

watercoolercodeON = """
from sevensegmentdisplay import encodeledstring, writeledstringautoscroll
import time

def phrases():
    for h in range(0, 100000):
        yield 10, "on %d0 sec" % h

@asyn.cancellable
async def setup():
    for t, ph in phrases():
        print(ph)
        encodeledstring(ph)
        t0 = time.time()
        while time.time() - t0 < t:
            writeledstringautoscroll()
            await asyn.sleep(0.1)
        
@asyn.cancellable
async def loop():
    pass
"""

watercoolercodeOFF = """
from sevensegmentdisplay import encodeledstring, writeledstringautoscroll
import time

def phrases():
    for h in range(0, 100000):
        yield 10, "off %d0 sec" % h

@asyn.cancellable
async def setup():
    for t, ph in phrases():
        print(ph)
        encodeledstring(ph)
        t0 = time.time()
        while time.time() - t0 < t:
            writeledstringautoscroll()
            await asyn.sleep(0.1)
        
@asyn.cancellable
async def loop():
    pass
"""

import time
def sendscriptletcode(mqtt, channel, code):
    nbytes = 50
    npieces = int(len(code)/nbytes+1)
    print("Sending code in", npieces, "pieces")
    for i in range(npieces):
        time.sleep(0.5)  # give it a chance to queue
        mqtt.publish("scriptlet1/code", "%03d/%03d]%s" % (i+1, npieces, code[i*nbytes:(i+1)*nbytes]))


Sent 54 lines (1338 bytes) to /main.py.


In [55]:
%sendtofile --append /main.py

from oled_driver import oledshowfattext
import time, machine

# used so we can break the cycle from the serial line
# on a reset button
if machine.wake_reason() != machine.PIN_WAKE:
    for i in range(10, 0, -1):
        oledshowfattext(["Reset", "Wake", str(i)])
        time.sleep(1)

oledshowfattext("DashButt")

import network
staif = network.WLAN(network.STA_IF)
staif.active(True)
staif.connect("DoESLiverpool", "decafbad00")
while not staif.isconnected():
    time.sleep(0.2)
oledshowfattext("MQTT")

from umqtt.robust import MQTTClient
mqttclient = MQTTClient(b"scriptlet1", "10.0.31.119")
mqttclient.connect()
mqttclient.publish(b"scriptlet1/ipbutton", staif.ifconfig()[0].encode())

# find the button state
pbutton = machine.Pin(13, machine.Pin.IN, machine.Pin.PULL_DOWN)
v1 = pbutton.value()
mqttclient.publish(b"scriptlet1/buttonstate", str(v1).encode())
pbutton.irq(trigger=machine.Pin.WAKE_HIGH if v1 == 0 else machine.Pin.WAKE_LOW, wake=machine.DEEPSLEEP)

oledshowfattext(["sending", str(v1)])
sendscriptletcode(mqttclient, "scriptlet1/code", watercoolercodeON if v1 == 1 else watercoolercodeOFF)
oledshowfattext(["done.", str(v1)])
time.sleep(2)

machine.deepsleep()


Sent 37 lines (1184 bytes) to /main.py.


In [56]:
%disconnect


attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.4-498-g5cd2c7f2e on 2018-09-08; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7efe59452240, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [9]:
help(rtc)

object <RTC> is of type RTC
  init -- <function>
  datetime -- <function>
  memory -- <function>


In [11]:
help("modules")

__main__          framebuf          socket            upip
_boot             gc                ssl               upip_utarfile
_onewire          hashlib           struct            upysh
_thread           heapq             sys               urandom
_webrepl          inisetup          time              ure
apa106            io                ubinascii         urequests
array             json              ucollections      uselect
binascii          machine           ucryptolib        usocket
btree             math              uctypes           ussl
builtins          micropython       uerrno            ustruct
cmath             neopixel          uhashlib          utime
collections       network           uhashlib          utimeq
dht               ntptime           uheapq            uzlib
ds18x20           onewire           uio               webrepl
errno             os                ujson             webrepl_setup
esp               random            umqtt/robust      websocket
esp32    

In [1]:
for l in open("/main.py"):
    print(l, end="")

# Needs: MicroPython_ESP32_psRAM_LoBo build

import time
import utime
from  machine import Pin, I2C
import machine
import ssd1306
import framebuf
import network
import ubinascii

# Definitions
ssid = 'DoESLiverpool'
password = 'decafbad00'

# Why did we wake up?
print(machine.wake_description())

# Setup OLED
rst = Pin(16, Pin.OUT)
rst.value(1)
scl = Pin(15, Pin.OUT, Pin.PULL_UP)
sda = Pin(4, Pin.OUT, Pin.PULL_UP)

i2c = I2C(scl=scl, sda=sda, freq=450000)

oled = ssd1306.SSD1306_I2C(128, 64, i2c, addr=0x3c)
 
# Gimme a welcome screen!
oled.fill(0)
oled.text('DoES Dash', 0, 0)
oled.show()
time.sleep(0.25)

# Set up networking
sta_if = network.WLAN(network.STA_IF)
 
oled.text('Wifi Connecting', 0, 10)
#oled.text(ssid, 0, 20)
oled.show()
 
if not sta_if.isconnected():
    print("Connecting to SSID", ssid)
    sta_if.active(True)
    sta_if.connect(ssid, password)
    while not sta_if.isconnected():
        pass
print("Connected! IP = ", sta_if.ifconfig()[0])

#oled.text("IP: " + sta_if.if

In [1]:
print(os.listdir("/"))

['remote', 'boot.py']


In [ ]:
watercoolercode = """
from sevensegmentdisplay import encodeledstring, writeledstringautoscroll
import time

def phrases():
    for h in range(100, 110):
        yield 6, "%d fresh" % h
    yield 3600, "fresh"
    for h in range(1, 4):
        yield 3600, "fresh %d hours" % h
    for h in range(1, 4):
        yield 3600, "fresh %d hours" % h
    yield 3600, "stale"
    for h in range(4, 24):
        yield 3600, "stale %d hours" % h
    for d in range(1, 100):
        yield 24*3600, "bad %d days old" % d

@asyn.cancellable
async def setup():
    for t, ph in phrases():
        print(ph)
        encodeledstring(ph)
        t0 = time.time()
        while time.time() - t0 < t:
            writeledstringautoscroll()
            await asyn.sleep(0.1)
        
@asyn.cancellable
async def loop():
    pass
"""

import time
def sendscriptletcode(mqtt, channel, code):
    nbytes = 50
    npieces = int(len(code)/nbytes+1)
    print("Sending code in", npieces, "pieces")
    for i in range(npieces):
        time.sleep(0.5)  # give it a chance to queue
        mqtt.publish("scriptlet1/code", "%03d/%03d]%s" % (i+1, npieces, code[i*nbytes:(i+1)*nbytes]))

# Needs: MicroPython_ESP32_psRAM_LoBo build

import time
import utime
from  machine import Pin, I2C
import machine
import ssd1306
import framebuf
import network
import ubinascii

# Definitions
ssid = 'DoESLiverpool'
password = 'decafbad00'

# Why did we wake up?
print(machine.wake_description())

# Setup OLED
rst = Pin(16, Pin.OUT)
rst.value(1)
scl = Pin(15, Pin.OUT, Pin.PULL_UP)
sda = Pin(4, Pin.OUT, Pin.PULL_UP)

i2c = I2C(scl=scl, sda=sda, freq=450000)

oled = ssd1306.SSD1306_I2C(128, 64, i2c, addr=0x3c)
 
# Gimme a welcome screen!
oled.fill(0)
oled.text('DoES Dash', 0, 0)
oled.show()
time.sleep(0.25)

# Set up networking
sta_if = network.WLAN(network.STA_IF)
 
oled.text('Wifi Connecting', 0, 10)
#oled.text(ssid, 0, 20)
oled.show()
 
if not sta_if.isconnected():
    print("Connecting to SSID", ssid)
    sta_if.active(True)
    sta_if.connect(ssid, password)
    while not sta_if.isconnected():
        pass
print("Connected! IP = ", sta_if.ifconfig()[0])

#oled.text("IP: " + sta_if.ifconfig()[0], 0, 30)
#oled.show()
#time.sleep(0.25)

rtc = machine.RTC()

# Sync RTC over NTP
if not rtc.synced():
    rtc.ntp_sync(server="hr.pool.ntp.org", tz="CET-1CEST")
    while not rtc.synced():
        pass
    print(utime.gmtime())
    print(utime.localtime())

# Send MQtt message
def conncb(task):
    print("[{}] Connected".format(task))

def disconncb(task):
    print("[{}] Disconnected".format(task))

def subscb(task):
    print("[{}] Subscribed".format(task))

def pubcb(pub):
    print("[{}] Published: {}".format(pub[0], pub[1]))

def datacb(msg):
    print("[{}] Data arrived from topic: {}, Message:\n".format(msg[0], msg[1]), msg[2])

mqtt = network.mqtt("does", "mqtt://10.0.29.187", user="user", password="pass", cleansession=True, connected_cb=conncb, disconnected_cb=disconncb, subscribed_cb=subscb, published_cb=pubcb, data_cb=datacb)

# secure connection requires more memory and may not work
#mqtts = network.mqtt("eclipse", "mqtts://iot.eclipse.org", cleansession=True, data_cb=datacb)

# mqtt over Websocket can also be used
# mqttws = network.mqtt("eclipse", "ws://iot.eclipse.org:80", cleansession=True, data_cb=datacb)
# mqttwss = network.mqtt("eclipse", "wss://iot.eclipse.org:80", cleansession=True, data_cb=datacb)

# Start the mqtt
print("Connecting to MQtt")
oled.text("Publishing Msg", 0, 20)
oled.show()

mqtt.start()

# Wait until status is: (1, 'Connected')
while mqtt.status()[0] < 2:
    print(mqtt.status())
    time.sleep(1)

#mqtt.subscribe('test')
mqtt.publish('button/' + ubinascii.hexlify(machine.unique_id()).decode('ascii'), 'woke up')
sendscriptletcode(mqtt, "scriptlet1/code", watercoolercode)

oled.text("Done. Sleeping", 0, 30)
oled.show()

i2c.deinit()

# Debug
time.sleep(1)

# Setup Sleep (wake on button press)
pin=machine.Pin(0)
pin.init(mode=pin.IN, pull=pin.PULL_UP)
rtc.wake_on_ext0(pin, 0)

print("Sleeping...")
machine.deepsleep()

